In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
from sklearn.model_selection import train_test_split

Extract the train.zip file

In [ ]:
import zipfile

z = zipfile.ZipFile('../input/dogs-vs-cats/train.zip')
z.extractall()

Import Necessary Libraries

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as transforms
import torch.optim as optim
import torchvision.models as models

from PIL import Image
import tqdm

Set the train image folder directory after extracting it

In [ ]:
train_imgDir = "../working/train"
# train_imgDir = 'train'
print(os.listdir(train_imgDir)[0].split('.')[0])

In [ ]:
imgFile, label = [],[]
for i in os.listdir(train_imgDir):
    imgFile.append(i)
    label.append(i.split('.')[0])

imgFile[:5], label[:5]

In [ ]:
train_df = pd.DataFrame(data=zip(imgFile,label), columns=['Filename','Label'])
train_df.head()

In [ ]:
train_df['Label'] = train_df['Label'].map({'dog':1,'cat':0})
train_df.head()

In [ ]:
tr, val = train_test_split(train_df, random_state=42, test_size=0.2, stratify=train_df['Label'])
print(tr['Label'].value_counts())
print(val['Label'].value_counts())

In [ ]:
tr['Filename'].iloc[0]

Create a custom dataset using torch's Dataset

In [ ]:
class DogsvsCats(Dataset):
    def __init__(self, img_dir, df, transform_apply=None, iftrain=True):
        self.imgDir = img_dir
        self.csvFile = df
        self.transform = transform_apply
        self.train = iftrain
    
    def __len__(self):
        return len(self.csvFile)
    
    def __getitem__(self, index):
        imgFile = os.path.join(self.imgDir,self.csvFile['Filename'].iloc[index])
        img = Image.open(imgFile)
        
        if self.transform:
            img = self.transform(img)
            
        if self.train:
            label = torch.tensor(self.csvFile['Label'].iloc[index])
            return img,label
        else:
            return img

Some tranformation apply to image

In [ ]:
# train_transforms = transforms.Compose([
#     transforms.Resize((224,224)),
#     transforms.ToTensor()
# ])

# train_dataset = DogsvsCats(train_imgDir,tr, train_transforms)

# means, stds = [],[]
# for img,_ in tqdm.notebook.tqdm(train_dataset):
#     means.append(torch.mean(img, dim=(1,2)).numpy())
#     stds.append(torch.std(img, dim=(1,2)).numpy())
    
# mean = torch.mean(torch.tensor(means), 0)
# std = torch.mean(torch.tensor(stds), 0)

In [ ]:
# mean, std

In [ ]:
train_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4884, 0.4551, 0.4170], std=[0.2256, 0.2210, 0.2214])
])

val_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4884, 0.4551, 0.4170], std=[0.2256, 0.2210, 0.2214])
])

create train dataset by calling custom dataset

In [ ]:
# DogsvsCats(train_imgDir,val, train_transforms).__getitem__(0)
train_dataset = DogsvsCats(train_imgDir,tr, train_transforms)
val_dataset = DogsvsCats(train_imgDir,val, val_transforms)

Make it to DataLoader

In [ ]:
BATCH_SIZE = 32
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
validation_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

Create the model for this task

In [ ]:
x = torch.randn((5,3,224,224))
x.shape

In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, in_fts, out_fts, k, s, p, bias=False):
        super(ConvBlock, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=in_fts, out_channels=out_fts, kernel_size=k, stride=s, padding=p, bias=bias),
            nn.BatchNorm2d(out_fts),
            nn.ReLU(inplace=True)
        )
        
    def forward(self, inp_img):
        return self.conv(inp_img)

class MyModel(nn.Module):
    def __init__(self, in_fts=3):
        super(MyModel, self).__init__()
        self.conv1 = ConvBlock(in_fts,64,7,2,3)
        self.conv2 = ConvBlock(64,128,3,1,1)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = ConvBlock(128,256,3,1,1)
        self.conv4 = ConvBlock(256,512,3,1,1)
        self.conv5 = ConvBlock(512,512,3,1,1)
        
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=(1,1))
        self.fc = nn.Sequential(
            nn.Linear(512,300),
            nn.Dropout(0.5),
            nn.Linear(300,300),
            nn.Dropout(0.5),
            nn.Linear(300,1),
            nn.Sigmoid()
        )
        

    def forward(self, input_img):
        N = input_img.shape[0]
        x = self.conv1(input_img)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.maxpool(x)
        x = self.conv3(x)
        x = self.maxpool(x)
        x = self.conv4(x)
        x = self.maxpool(x)
        x = self.conv5(x)
        x = self.avgpool(x)
        x = torch.flatten(x,1)
        x = self.fc(x)
        
        return x

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = MyModel().to(device)
x = x.to(device)
# model = models.resnet101(pretrained=False).to(device)
print(model(x).shape)
# print(model(x))

Define loss and optimizer function

In [ ]:
LR = 1e-1
loss_fn = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=LR)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=1, verbose=True)

Train step

In [ ]:
EPOCHS = 30
for e in range(EPOCHS):
    loop = tqdm.notebook.tqdm(train_dataloader)
    loop.set_description(f"{e+1}/{EPOCHS}")
    total_train_loss, total_train_accuracy = 0,0
    iteration = 1
    for img, label in loop:
        img = img.to(device)
        label = label.to(device)
        output = model(img)

        L = loss_fn(output, label.float().unsqueeze(1))
        total_train_loss += L.item()
        
        pred = []
        for i in output:
            if i.item() >= 0.5:
                pred.append(1)
            else:
                pred.append(0)
                
        accuracy = torch.mean((torch.tensor(pred).to(device) == label).float())
        total_train_accuracy += accuracy.item()
        
        optimizer.zero_grad()
        L.backward()
        optimizer.step()
        
            
        loop.set_postfix({'Train Loss':total_train_loss/iteration,'Train Accuracy':total_train_accuracy/iteration})
        iteration += 1
        
    if (e+1)%1 == 0:
        iteration = 1
        loop = tqdm.notebook.tqdm(validation_dataloader)
        total_val_loss, total_val_accuracy = 0,0
        model.eval()
        with torch.no_grad():
            for img,label in loop:
                img = img.to(device)
                label = label.to(device)
                output = model(img)
                
                L = loss_fn(output, label.float().unsqueeze(1))
                total_val_loss += L.item()
                
                pred = []
                for i in output:
                    if i.item() >= 0.5:
                        pred.append(1)
                    else:
                        pred.append(0)
                        
                accuracy = torch.mean((torch.tensor(pred).to(device) == label).float())
                total_val_accuracy += accuracy.item()
                
                loop.set_postfix({'Val Loss':total_val_loss/iteration,'Val Accuracy':total_val_accuracy/iteration})
                iteration += 1
        
        model.train()
        scheduler.step(total_val_loss/iteration)

Save the entire Model.
Ref: https://pytorch.org/tutorials/recipes/recipes/saving_and_loading_models_for_inference.html

In [ ]:
save_path = "MyModel.pt"

torch.save(model, save_path)

In [ ]:
# To cross check whether the model 

temp_model = torch.load("../working/MyModel.pt")
# temp_model = torch.load("MyModel.pt")
temp_model = temp_model.to(device)
temp_model.eval()
iteration = 1
loop = tqdm.notebook.tqdm(validation_dataloader)
total_val_loss, total_val_accuracy = 0,0

with torch.no_grad():
    for img,label in loop:
        img = img.to(device)
        label = label.to(device)
        output = temp_model(img)

        L = loss_fn(output, label.float().unsqueeze(1))
        total_val_loss += L.item()

        pred = []
        for i in output:
            if i.item() >= 0.5:
                pred.append(1)
            else:
                pred.append(0)

        accuracy = torch.mean((torch.tensor(pred).to(device) == label).float())
        total_val_accuracy += accuracy.item()

        loop.set_postfix({'Val Loss':total_val_loss/iteration,'Val Accuracy':total_val_accuracy/iteration})
        iteration += 1

In [ ]:
import shutil
shutil.rmtree("../working/train")

For Testing...

In [ ]:
z = zipfile.ZipFile('../input/dogs-vs-cats/test1.zip')
z.extractall()

Make it image Filename list

In [ ]:
test_imgDir = "../working/test1"
imgFile = []
for i in os.listdir(test_imgDir):
    imgFile.append(i)

imgFile[:5]

Convert into pandas DataFrame

In [ ]:
test_df = pd.DataFrame(data=imgFile, columns=['Filename'])
test_df.head()

Read Dataset and DataLoader

In [ ]:
test_dataset = DogsvsCats(test_imgDir,test_df, val_transforms, False)

In [ ]:
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

Predict the output

In [ ]:
temp_model.eval()
loop = tqdm.notebook.tqdm(test_dataloader)
test_pred = []

with torch.no_grad():
    for img in loop:
        img = img.to(device)
        output = temp_model(img)

        pred = []
        for i in output:
            if i.item() >= 0.5:
                pred.append(1)
            else:
                pred.append(0)
                
        test_pred.extend(pred)

Make a submission file for id and predict label

In [ ]:
submission = pd.DataFrame(data=zip(imgFile,test_pred),columns=["id","label"])
submission.head()

Removal of extension filename .jpg

In [ ]:
submission["id"] = submission["id"].apply(lambda x: int(x.split('.')[0]))
submission.head()

Sort it to the ascending order

In [ ]:
submission.sort_values("id", axis=0, ascending=True, inplace=True)

In [ ]:
submission.reset_index()
submission.head()

In [ ]:
submission.to_csv("submission_01.csv", index=False)
print("Done..")

In [ ]:
shutil.rmtree("../working/test1")